In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.geom2d import SplineGeometry
from ngsolve.internal import *
import numpy as np


In [2]:
mesh = Mesh("mesh.vol")


# --- Load your snapshots (check shapes!) ---
U_snap_full  = np.load("U_snap_freq60.0Hz.npy.npz")["arr_0"]   # shape (n_steps, ndofs*dim) or (n_steps, ndofs, 2)
Phi_snap_full= np.load("Phi_snap_freq60.0Hz.npy.npz")["arr_0"] # shape (n_steps, ndofs)
T_snap_full = np.load("times_freq60.0Hz.npy.npz")["arr_0"]            # optional: times, to recover dt_NS

U_snap = U_snap_full[::3]
Phi_snap = Phi_snap_full[::3]
T_snap = T_snap_full[::3]
n_steps = Phi_snap.shape[0]

# Suppose your velocity space Vv and pressure space Qp are same as in NS sim
Vv = VectorH1(mesh, order=2)
Qp = H1(mesh, order=1)
V1 = H1(mesh, order=1,dirichlet="Gas_Inlet|Liq_Inlet")              # scalar fields: phi, alpha, concentration
c_trial, v_test = V1.TnT()

# --- GridFunctions for replayed flow & fields ---
u_gfu   = GridFunction(Vv)    # precomputed velocity
phi_gfu = GridFunction(V1)    # precomputed level-set (or any phase indicator)
c       = GridFunction(V1)    # unknown: dissolved CO2 concentration (mol/m^3)







# DOF counts
n_vel_dofs = Vv.ndof
n_p_dofs   = Qp.ndof

def set_state(step):
    sol = U_snap[step]   # shape (ndofs_total,)
    u_part = sol[:n_vel_dofs]
    p_part = sol[n_vel_dofs:]  # not used here, but you can save it

    # assign only velocity into u_gfu
    u_gfu.vec.FV().NumPy()[:] = u_part
    phi_gfu.vec.FV().NumPy()[:] = Phi_snap[step]

from ngsolve import exp   # <-- this is the right one

set_state(0)
g_in = CoefficientFunction(4.9)
c.Set(g_in, definedon=mesh.Boundaries("Gas_Inlet"))

In [3]:
Draw(c,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [4]:
# --- Model params (same as you had; fill your real numbers) ---
dt       = CoefficientFunction(5e-4)   # your NS timestep
D_L      = Parameter(1.5e-9)           # tweak lower if MEA viscous
D_g      = Parameter(1.5e-5)
k_CO2    = Parameter(1.0)              
R_gas    = 8.3145
T_const  = Parameter(315.0)            # 42 °C
phi_MEA  = Parameter(0.094)            # mole fraction
theta    = Parameter(0.30)
c_MEA    = Parameter(4000.0)           # mol/m³


r_T  = exp(20.54 - 5612.91/T_const)
c_eq = (30.96 - 10584.0/T_const - 7.1878*phi_MEA) \
       * (theta*theta)/((1.0-theta)**2) * 1.0/(k_CO2*R_gas*T_const)

# Map Allen–Cahn φ → volume fraction αL (no extra smoothing)
alpha_L = 0.5*(1.0 + phi_gfu)     # if φ∈[-1,1]; else alpha_L = phi_gfu

# One-fluid harmonic diffusivity (pure CF expression)
D_harm = (D_L*D_g) / ( alpha_L*D_g + (1.0-alpha_L)*D_L )

# --- Weak forms (keep all grad(...) inline) ---
a = BilinearForm(V1)
a += (1.0/dt) * c_trial * v_test * dx
a += D_harm * grad(c_trial) * grad(v_test) * dx

# Advection in conservative form: (u c, ∇v)
a += InnerProduct(u_gfu, grad(v_test)) * c_trial * dx

# Implicit part of reaction: + r c_MEA αL c^{n+1}
a += (r_T * c_MEA * alpha_L) * c_trial * v_test * dx

L = LinearForm(V1)
L += (1.0/dt) * c * v_test * dx

# ---- Interfacial correction term Γ lagged in c (linear) ----
# Γ(c^n) = -D_harm * c^n * (1-k) / (αL k + (1-αL)) * ∇αL
# and ∇αL = 0.5 ∇φ  ; keep grad(...) inline:
denom_cf = (alpha_L*k_CO2 + (1.0 - alpha_L))
Gamma_dot_gradv = InnerProduct(
    - D_harm * (c * (1.0 - k_CO2)) / denom_cf * (0.5*grad(phi_gfu)),
    grad(v_test)
)
L += Gamma_dot_gradv * dx

# RHS of reaction: r c_MEA αL c_eq
L += (r_T * c_MEA * alpha_L * c_eq) * v_test * dx


In [5]:
# --- Inverse options (adjust to your solver setup) ---

# optional: storage for results
results = []
# Initial guess with inlet concentration
c.Set(g_in, definedon=mesh.Boundaries("Gas_Inlet"))

for step in range(n_steps):
    set_state(step)

    a.Assemble()
    L.Assemble()

    res = L.vec.CreateVector()
    res.data = L.vec - a.mat * c.vec     # residual

    c.vec.data += a.mat.Inverse(V1.FreeDofs()) * res

    results.append(c.vec.FV().NumPy().copy())

    if step % 10 == 0:
        print(f"Step {step}")
        Draw(u_gfu,mesh)
        Draw(c, mesh)


Step 0


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

Step 10


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

...

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

Step 40


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [6]:
print("U_snap shape:", U_snap.shape)
print("Phi_snap shape:", Phi_snap.shape)
print("n_vel_dofs:", n_vel_dofs, "n_p_dofs:", n_p_dofs, {n_vel_dofs+n_p_dofs})


U_snap shape: (50, 174723)
Phi_snap shape: (50, 14789)
n_vel_dofs: 116482 n_p_dofs: 14789 {131271}


In [7]:
print("U_snap shape:", U_snap.shape)
for s in [0, 1, 2, 10, 149]:
    print(f"step {s}: max|u| = {np.max(np.abs(U_snap[s,:]))}")


U_snap shape: (50, 174723)
step 0: max|u| = 41.42623402439366
step 1: max|u| = 41.154696246880306
step 2: max|u| = 47.14136728005469
step 10: max|u| = 81.99926435793235


IndexError: index 149 is out of bounds for axis 0 with size 50